In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
import re
import datetime
import time
import pickle
import mlflow
import mlflow.sklearn
import pymysql
import os

In [2]:
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Ludi24',
                         db='journaux')
# create cursor
cursor = connection.cursor()

query_read = "Select * from articles  where jour_publication<=date('2021-04-20') and  jour_publication>=date('2020-9-10')"
cursor.execute(query_read)
raw = cursor.fetchall()
connection.commit()

In [3]:
df = pd.DataFrame(raw, columns =["id_article","titre" ,"jour_publication" ,"categorie","journal"  ])
cursor.close()
connection.close()
df

,id_article,titre,jour_publication,categorie,journal
0,1,"Pour les galeries d’art, une situation moins s...",2021-02-01,economie,Le Monde
1,2,Quand la masturbation provoque une hémorragie ...,2021-02-01,inconnue,Le Monde
2,3,Les pistes de la France pour améliorer les ter...,2021-02-01,economie,Le Monde
3,4,Facebook et Apple se livrent une guerre ouverte,2021-02-01,economie,Le Monde
4,5,"Kent Walker : « Google agit selon les lois, et...",2021-02-01,pixels,Le Monde
...,...,...,...,...,...
48445,160810,Covid-19 : la tentation du relâchement,2021-04-20,checknews,Libération
48446,160811,Mort d’Idriss Déby : le chaos va-t-il se rajou...,2021-04-20,moyen-orient,Libération
48447,160812,"Manchester City, Arsenal, Liverpool, Tottenham...",2021-04-20,football,Libération
48448,160813,Derek Chauvin déclaré coupable du meurtre de G...,2021-04-20,amérique,Libération


In [4]:
df.categorie.value_counts()[:40]

société             6621
international       5690
economie            2884
culture             2809
monde               2781
politique           2775
people              2194
idees               2125
societe             1897
planete             1644
afrique             1535
idées et débats     1151
actualité           1141
livres               893
sport                848
sciences             708
m-le-mag             655
santé                587
inconnue             559
checknews            544
pixels               495
royautes             446
m-styles             433
vecu                 380
europe               380
environnement        361
disparitions         348
sports               347
les-decodeurs        312
argent               306
lifestyle            297
m-perso              247
amerique du nord     242
musique              236
campus               234
emploi               193
police-justice       189
tribunes             177
cinéma               140
education            122


# Preprocessing Nettoyage

In [5]:
len(df.groupby('jour_publication'))

223

In [6]:
# récupération dans le package des stop-mots
import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('french')
stop_words.extend(["c'est","j'ai","a","plus","contre","après", "d'un","d'une","entre","ans","deux","veut","comme",
"va","trois","sous","faut","n'est","cinq","leurs","doit","qu'il","peut","n'a","mis","six","cette","j'ai","-","s'est","dit","dont"])

In [7]:
from nltk.stem.snowball import FrenchStemmer #import utilisé pour raciniser les mots , finalement pas utilisé

# fonction qui enlève les caractères spéciaux
def nettoyage(text):
    stemmer = FrenchStemmer()
    text = str(text).lower() # mettre les mots en minuscule
    text = re.sub(r"[.,\!\?\%\(\)\/\"]", "", text)  # Retrait les caractères spéciaux :
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\d", "", text) 
    text = re.sub(r"\-", "", text) 
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\»", "", text) 
    text = re.sub(r"\«", "", text)
    text = re.sub(r"\’", " ", text)
    text = text.split()
    les_mots = ""
    for mot in text:
        a_ajouter = stemmer.stem(mot)
        if a_ajouter not in stop_words:
            les_mots = les_mots + " "+ a_ajouter
    return les_mots

In [8]:
def encode_journal(x):
        if x =="Le Monde":
            val = 1
        elif x== "Libération":
            val = 2
        elif x =="L'Express":
            val = 3
        elif x == "Closer":
            val = 4
        return val

In [9]:
df.titre = df.titre.map(lambda x : nettoyage(x))

In [10]:
df.journal = df.journal.map(lambda x : encode_journal(x))
df

,id_article,titre,jour_publication,categorie,journal
0,1,galer art situat moin sombr redout,2021-02-01,economie,1
1,2,quand masturb provoqu hémorrag méning,2021-02-01,inconnue,1
2,3,pist franc amélior term accord libreéchang me...,2021-02-01,economie,1
3,4,facebook apple livrent guerr ouvert,2021-02-01,economie,1
4,5,kent walk googl agit selon lois non selon pol...,2021-02-01,pixels,1
...,...,...,...,...,...
48445,160810,covid tentat relâch,2021-04-20,checknews,2
48446,160811,mort idriss déby chaos vatil rajout chaos,2021-04-20,moyen-orient,2
48447,160812,manchest city arsenal liverpool tottenham man...,2021-04-20,football,2
48448,160813,derek chauvin déclar coupabl meurtr georg floyd,2021-04-20,amérique,2


## Split

In [11]:
X = df.titre
y = df['journal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))

(33915,) (14535,) (33915,) (14535,)


In [12]:
X_train

46472                  espac coworking aid étudi sort isol
18460        karach rumeur accus mensonger dénonc balladur
46806     téléthon pres million d'euros récolt malgr co...
27180                  tchad grev enseign dan écol publiqu
8201                                 non mort vot jo biden
                               ...                        
21243      environ futur del d'écocid seratil vrai efficac
45891     jo suspens traval parc georgesvalbon premi ét...
42613     offre menus végétarien étend dan cantin heurt...
43567                                 givr gel hydroalcool
2732      obtent démocrat égal sieg sénat géorg permett...
Name: titre, Length: 33915, dtype: object

In [13]:
y_train.value_counts()

1    15356
2     8492
3     7473
4     2594
Name: journal, dtype: int64

In [14]:
y_test.value_counts()

1    6564
2    3600
3    3339
4    1032
Name: journal, dtype: int64

In [15]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

# Sur-échantillonnage
rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(np.array(X_train).reshape(-1,1), y_train)

In [16]:
X_train = X_ro.reshape(-1,)
y_train = y_ro

In [17]:
print(y_train.value_counts().apply(lambda x: x/len(y_train)))
print(y_test.value_counts().apply(lambda x: x/len(y_test)))

1    0.25
2    0.25
3    0.25
4    0.25
Name: journal, dtype: float64
1    0.451600
2    0.247678
3    0.229721
4    0.071001
Name: journal, dtype: float64


## PREPROCESSING Vectorisation et pondération

from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer='word')
corpus = X_train.to_list()
vectorizer.fit(corpus)
pipe = Pipeline([('count', CountVectorizer(vocabulary= vectorizer.get_feature_names())),
                 ('tfid', TfidfTransformer())]).fit(corpus)

df_tfidf = pipe.transform(corpus).toarray()
print(np.shape(df_tfidf))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word')
tfidf.fit(X_train)

TfidfVectorizer()

In [19]:
X_train_word_features = tfidf.transform(X_train)
test_features = tfidf.transform(X_test)

In [20]:
print(len(tfidf.get_feature_names()))

22618


In [21]:
X_train_word_features.shape

(61424, 22618)

In [ ]:
file = 'Flask_journaux/src/models_pickle/file_tfidf.pkl' 
pickle.dump(tfidf, open(file, 'wb'))

## 1.1 Modèle KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix ,classification_report
from sklearn.metrics import precision_score , recall_score, f1_score , r2_score , accuracy_score

In [24]:
with mlflow.start_run():
    print("Training KNN...")
    knn = KNeighborsClassifier()    
    knn.fit(X_train_word_features, y_train)
    pred_knn = knn.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_knn))
    print('Precision', precision_score(y_test, pred_knn ,average='weighted'))
    print('Recall', recall_score(y_test, pred_knn,average='weighted'))
    print('f1-score', f1_score(y_test, pred_knn,average='weighted'))
    mlflow.sklearn.log_model(knn,'KNN')
    mlflow.log_param('algorithm','KNN')
    mlflow.log_param('best_params',knn.get_params(deep=True))
    mlflow.log_param('cross_val',cross_val_score(knn, X_train_word_features, y_train, cv=5))
    mlflow.log_metric("score",accuracy_score(y_test, pred_knn))
    print("Fin de traitement")

2021/05/21 10:55:33 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Training KNN...
Accurancy 0.48434812521499826
Precision 0.4836848299580491
Recall 0.48434812521499826
f1-score 0.48345566381233496
Fin de traitement


In [25]:
knn.get_params(deep=True)

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [26]:
confusion_matrix(y_test, pred_knn)

array([[3647, 1590, 1136,  191],
       [1462, 1487,  534,  117],
       [1325,  714, 1202,   98],
       [ 131,  117,   80,  704]], dtype=int64)

In [27]:
print(classification_report(y_test, pred_knn))

              precision    recall  f1-score   support

           1       0.56      0.56      0.56      6564
           2       0.38      0.41      0.40      3600
           3       0.41      0.36      0.38      3339
           4       0.63      0.68      0.66      1032

    accuracy                           0.48     14535
   macro avg       0.49      0.50      0.50     14535
weighted avg       0.48      0.48      0.48     14535



In [28]:
file = 'Flask_journaux/src/models_pickle/file_model_fitted_Knn.pkl' 
pickle.dump(knn, open(file, 'wb'))

## 1.2. Modèle Random Forest

In [29]:
with mlflow.start_run():
    print("Training Randomforest...")
    rf = RandomForestClassifier()
    rf.fit(X_train_word_features, y_train)
    pred_rf = rf.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_rf))
    print('Precision', precision_score(y_test, pred_rf ,average='weighted'))
    print('Recall', recall_score(y_test, pred_rf,average='weighted'))
    print('f1-score', f1_score(y_test, pred_rf,average='weighted'))
    mlflow.sklearn.log_model(rf,'Random Forest')
    mlflow.log_param('algorithm','RandomForest')
    mlflow.log_param('best_params',rf.get_params(deep=True))
    mlflow.log_param('cross_val',cross_val_score(rf, X_train_word_features, y_train, cv=5))
    mlflow.log_metric("score",accuracy_score(y_test, pred_rf))
    print("Fin de traitement")

Training Randomforest...
Accurancy 0.565875472996216
Precision 0.5604511736785047
Recall 0.565875472996216
f1-score 0.5596065038146303
Fin de traitement


In [29]:
rf.get_params(deep=True)

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

model_forest = RandomForestClassifier()
n_estimators = [10,100,200]
max_features = ['sqrt', 'log2']
params_forest = dict(n_estimators = n_estimators, max_features = max_features)

#model = RandomizedSearchCV(estimator = model_forest, param_distributions = param, n_jobs=-1, cv=6, scoring='f1',error_score=0)
model = RandomForestClassifier(n_estimators = 10, max_features= 'sqrt', n_jobs=-1)
model.fit(X_train_word_features, y_train)
pred = model.predict(test_features)
data = {'id_run':datetime.datetime.today() , 'model':str(model), 'hyperparameters':model.get_params(deep=True), 'f1':f1_score(y_test, pred,average='weighted')}
with open('hyperparametres.txt', 'w') as outfile:
    json.dump(data, outfile)

In [30]:
# calcul de la matrise de confusion 
confusion_matrix(y_test, pred_rf)

array([[1154,  768,  412,  204],
       [ 388, 1523,  311,  152],
       [ 507,  574,  998,  160],
       [  75,  116,   37, 2349]], dtype=int64)

In [31]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           1       0.54      0.45      0.50      2538
           2       0.51      0.64      0.57      2374
           3       0.57      0.45      0.50      2239
           4       0.82      0.91      0.86      2577

    accuracy                           0.62      9728
   macro avg       0.61      0.61      0.61      9728
weighted avg       0.61      0.62      0.61      9728



## 1.3. MODELE SVM

In [30]:
with mlflow.start_run():
    print("training SVM...")
    svm = SVC()
    svm.fit(X_train_word_features, y_train)
    pred_svm = svm.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_svm))
    print('Precision', precision_score(y_test, pred_svm ,average='weighted'))
    print('Recall', recall_score(y_test, pred_svm,average='weighted'))
    print('f1-score', f1_score(y_test, pred_svm,average='weighted'))
    mlflow.sklearn.log_model(svm,'SVM model')
    mlflow.log_param('algorithm','SVM')
    mlflow.log_param('best_params',svm.get_params(deep=True))
    mlflow.log_param('cross_val',cross_val_score(svm, X_train_word_features, y_train, cv=5))
    mlflow.log_metric("score",accuracy_score(y_test, pred_svm))
    print("Fin de traitement")

training SVM...
Accurancy 0.5893360853113175
Precision 0.5841586537927693
Recall 0.5893360853113175
f1-score 0.5786601926828402
Fin de traitement


In [51]:
# avec probability true  pour avoir les probas
with mlflow.start_run():
    print("training SVM...")
    svm = SVC(probability=True) 
    svm.fit(X_train_word_features, y_train)
    pred_svm = svm.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_svm))
    print('Precision', precision_score(y_test, pred_svm ,average='weighted'))
    print('Recall', recall_score(y_test, pred_svm,average='weighted'))
    print('f1-score', f1_score(y_test, pred_svm,average='weighted'))
    mlflow.sklearn.log_model(svm,'SVM model')
    mlflow.log_param('algorithm','SVM')
    mlflow.log_param('best_params',svm.get_params(deep=True))
    #mlflow.log_param('cross_val',cross_val_score(svm, X_train_word_features, y_train, cv=5))
    mlflow.log_metric("score",accuracy_score(y_test, pred_svm,average='weighted'))
    print("Fin de traitement")

training SVM...
Accurancy 0.6408305921052632
Precision 0.6455588892625438
Recall 0.6408305921052632
f1-score 0.6408707013380684
Fin de traitement


In [49]:
svm.predict_proba(test_features)

array([[3.83300759e-03, 4.04580142e-03, 6.63743919e-03, 9.85483752e-01],
       [4.84986786e-09, 5.52340215e-09, 3.45009503e-09, 9.99999986e-01],
       [6.92627957e-01, 1.81541323e-01, 1.25049384e-01, 7.81336424e-04],
       ...,
       [2.87580736e-01, 2.85483834e-01, 3.95250031e-01, 3.16853979e-02],
       [2.62916533e-01, 2.00245395e-01, 5.32837338e-01, 4.00073289e-03],
       [3.20500886e-01, 3.56217833e-01, 3.01747443e-01, 2.15338379e-02]])

In [33]:
svm.get_params(deep=True)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [34]:
confusion_matrix(y_test, pred_svm)

array([[1500,  594,  374,   70],
       [ 697, 1296,  317,   64],
       [ 675,  426, 1075,   63],
       [  95,   83,   36, 2363]], dtype=int64)

In [35]:
print(classification_report(y_test, pred_svm))

              precision    recall  f1-score   support

           1       0.51      0.59      0.54      2538
           2       0.54      0.55      0.54      2374
           3       0.60      0.48      0.53      2239
           4       0.92      0.92      0.92      2577

    accuracy                           0.64      9728
   macro avg       0.64      0.63      0.64      9728
weighted avg       0.65      0.64      0.64      9728



## 1.4. MODELE NAIVE BAYES

In [52]:
with mlflow.start_run():
    print("training Naive bayes...")
    mnb = MultinomialNB()
    mnb.fit(X_train_word_features.toarray(), y_train)
    pred_mnb = mnb.predict(test_features.toarray())
    print('Accurancy', accuracy_score(y_test, pred_mnb))
    print('Precision', precision_score(y_test, pred_mnb ,average='weighted'))
    print('Recall', recall_score(y_test, pred_mnb,average='weighted'))
    print('f1-score', f1_score(y_test, pred_mnb,average='weighted'))
    mlflow.sklearn.log_model(mnb,'Naive_bayes')
    mlflow.log_param('algorithm','Naive_bayes')
    mlflow.log_param('best_params',mnb.get_params(deep=True))
    mlflow.log_param('cross_val',cross_val_score(mnb, X_train_word_features, y_train, cv=5))
    mlflow.log_metric("score",accuracy_score(y_test, pred_mnb))
    print("Fin de traitement")

training Naive bayes...
Accurancy 0.5136838314728124
Precision 0.5117851979488818
Recall 0.5136838314728124
f1-score 0.5023671774103569
Fin de traitement


In [37]:
confusion_matrix(y_test, pred_mnb)

array([[1566,  343,  457,  172],
       [ 877,  847,  422,  228],
       [ 728,  260, 1111,  140],
       [  67,   19,   30, 2461]], dtype=int64)

In [38]:
print(classification_report(y_test, pred_mnb))

              precision    recall  f1-score   support

           1       0.48      0.62      0.54      2538
           2       0.58      0.36      0.44      2374
           3       0.55      0.50      0.52      2239
           4       0.82      0.95      0.88      2577

    accuracy                           0.62      9728
   macro avg       0.61      0.61      0.60      9728
weighted avg       0.61      0.62      0.60      9728



## 2.1. Hyperparamètres & Modèle KNN

In [31]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

In [32]:
# recherche via randomsied serach knn
with mlflow.start_run():
    print("Tuning KNN...")
    knn = KNeighborsClassifier()
    n_neighbors = range(1, 21, 2)
    weights = ['uniform', 'distance']
    metric = ['euclidean', 'manhattan', 'minkowski']

    # define grid search
    params_knn = dict(n_neighbors = n_neighbors , weights = weights, metric = metric)
    #cv_knn = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    random_search_knn = RandomizedSearchCV(estimator=knn, param_distributions = params_knn, n_jobs=-1, cv=5, scoring='accuracy',error_score=0)
    random_result_knn = random_search_knn.fit(X_train_word_features, y_train)

    # summarize results
    print("Best: %f using %s" % (random_result_knn.best_score_, random_result_knn.best_params_))
    mlflow.sklearn.log_model(KNeighborsClassifier(), "journaux")
    mlflow.log_param('algorithm','KNN randomisedSearch')
    mlflow.log_param('best_params',random_result_knn.best_params_)
    mlflow.log_metric("best_score", random_result_knn.best_score_)
    print("Fin de traitement")

Tuning KNN...
Best: 0.794781 using {'weights': 'distance', 'n_neighbors': 7, 'metric': 'euclidean'}
Fin de traitement


In [ ]:
# entrainement du meilleur set d'hyperparamètres KNN
with mlflow.start_run():
    print("Traning best model Knn...")
    knn_opti = KNeighborsClassifier(n_neighbors = 15 , weights = 'distance', metric ='euclidean')
    #scores_cross_val_knn= cross_val_score(knn_opti,X_train_word_features,y_train, cv=5 )
    knn_opti.fit(X_train_word_features, y_train)
    pred_knn_opti = knn_opti.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_knn_opti))
    print('Precision', precision_score(y_test, pred_knn_opti ,average='weighted'))
    print('Recall', recall_score(y_test, pred_knn_opti,average='weighted'))
    print('f1-score', f1_score(y_test, pred_knn_opti,average='weighted'))
    mlflow.log_param('best_params',knn_opti.get_params(deep=True))
    mlflow.log_metric("score", accuracy_score(y_test, pred_knn_opti,average='weighted'))
    mlflow.sklearn.log_model(KNeighborsClassifier(), "journaux")
    #mlflow.log_param('cross_val',scores_cross_val_knn)
    mlflow.log_param('algorithm','KNN tuned')
    print("Fin de traitement")

In [37]:
file_knn = 'Flask_journaux/src/models_pickle/file_model_fitted_knn_opti.pkl' 
pickle.dump(knn_opti, open(file_knn, 'wb'))

## 2.2 Hyperparamètres & Random forest

In [ ]:
# recherche meilleur set hyperparamètres RandonForest
with mlflow.start_run():
    print("Tuning Randomforest...")
    model_forest = RandomForestClassifier()
    n_estimators = [10,100,200]
    max_features = ['sqrt', 'log2']
    params_forest = dict(n_estimators = n_estimators, max_features = max_features)
    #cv_forest = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    random_search_forest = RandomizedSearchCV(estimator = model_forest, param_distributions = params_forest, n_jobs=-1, cv=5, scoring='accuracy',error_score=0)
    random_result_forest = random_search_forest.fit(X_train_word_features, y_train)
    print("Best: %f using %s" % (random_result_forest.best_score_, random_result_forest.best_params_))
    mlflow.log_param('best_params',random_result_forest.best_params_)
    mlflow.log_metric("best_score", random_result_forest.best_score_)
    mlflow.sklearn.log_model(RandomForestClassifier(), "journaux")
    mlflow.log_param('algorithm', 'RandomForest randomisedSearch')
    print("Fin de traitement")

In [55]:
with mlflow.start_run():
    print("Training best model Randomforest...")
    rf_opti = RandomForestClassifier(n_estimators=200,max_features='log2')
    #scores_cross_val_rf= cross_val_score(rf_opti,X_train_word_features, y_train, cv=5)
    rf_opti.fit(X_train_word_features, y_train)
    pred_rf_opti = rf_opti.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_rf_opti,average='weighted'))
    print('Precision', precision_score(y_test, pred_rf_opti ,average='weighted'))
    print('Recall', recall_score(y_test, pred_rf_opti,average='weighted'))
    print('f1-score', f1_score(y_test, pred_rf_opti,average='weighted'))
    mlflow.log_param('best_params',rf_opti.get_params())
    mlflow.log_metric("score", accuracy_score(y_test, pred_rf_opti,average='weighted'))
    mlflow.sklearn.log_model(RandomForestClassifier(), "journaux")
    mlflow.log_param('algorithm', 'RandomForest')
    #mlflow.log_param('cross_val',scores_cross_val_rf)
    print("Fin de traitement")

Training best model Randomforest...
Accurancy 0.5823185414516684
Precision 0.5821308944016204
Recall 0.5823185414516684
f1-score 0.576203170576184


TypeError: accuracy_score() got an unexpected keyword argument 'average'

In [ ]:
y_score = rf_opti.score(y_test,pred_rf_opti)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
from sklearn.metrics import roc_curve ,auc
fpr, tpr, threshold = roc_curve(y_test,pred_rf_opti)
roc_auc = auc(fpr, tpr)
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [39]:
file_rf = 'Flask_journaux/src/models_pickle/file_model_fitted_RandonForest_opti.pkl' 
pickle.dump(rf_opti, open(file_rf, 'wb'))

##  2.3. Hyperparamètres & MODELE SVM

In [ ]:
with mlflow.start_run(nested=True):
    print("Tuning SVM...")
    svm = SVC()
    kernel = ['poly', 'rbf', 'sigmoid']
    C = [50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    # define grid search
    params_svm = dict(kernel=kernel,C=C,gamma=gamma)
    #cv_svm = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    random_search_SVM = RandomizedSearchCV(estimator=svm, param_distributions=params_svm, n_jobs=-1, cv=5, scoring='accuracy',error_score=0)
    random_result_SVM = random_search_SVM.fit(X_train_word_features, y_train)
    # summarize results
    print("Best: %f using %s" % (random_result_SVM.best_score_, random_result_SVM.best_params_))
    mlflow.log_param('best_params',random_result_SVM.best_params_)
    mlflow.log_param('algorithm','SVM randomizedSearchCV')
    mlflow.log_metric("best_score", random_result_SVM.best_score_)
    mlflow.sklearn.log_model(SVC(), "journaux")
    print("Fin de traitement")

Tuning SVM...


In [31]:
# copie de la cellule du haut
with mlflow.start_run(nested=True):
    print("Tuning SVM...")
    svm = SVC()
    kernel = ['poly', 'rbf', 'sigmoid']
    C = [50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    # define grid search
    params_svm = dict(kernel=kernel,C=C,gamma=gamma)
    cv_svm = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    random_search_SVM = RandomizedSearchCV(estimator=svm, param_distributions=params_svm, n_jobs=-1, cv=cv_svm, scoring='accuracy',error_score=0)
    random_result_SVM = random_search_SVM.fit(X_train_word_features, y_train)
    # summarize results
    print("Best: %f using %s" % (random_result_SVM.best_score_, random_result_SVM.best_params_))
    mlflow.log_param('best_params',random_result_SVM.best_params_)
    mlflow.log_param('algorithm','SVM randomizedSearchCV')
    mlflow.log_metric("best_score", random_result_SVM.best_score_)
    mlflow.sklearn.log_model(SVC(), "journaux")
    print("Fin de traitement")

Tuning SVM...
Best: 0.631419 using {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}
Fin de traitement


In [32]:
# donne les valeurs des hyperparametres et le score ici accuracy
a_enregistrer = random_result_SVM.cv_results_
a_enregistrer

{'mean_fit_time': array([322.18364176, 117.40339882, 333.18399922, 157.41959755,
        123.95367841, 110.03318853,  84.28457875, 234.59466761,
        289.0349294 , 118.2553923 ]),
 'std_fit_time': array([ 5.11694405,  2.16924064, 10.25318381,  6.4141064 ,  2.89853473,
         2.72571445,  2.24538314, 34.18457842,  6.35135466,  3.76527422]),
 'mean_score_time': array([7.04833902, 7.18716965, 7.24186418, 6.54423445, 7.03660782,
        6.84834101, 5.74491012, 4.65829961, 6.20478531, 6.44726063]),
 'std_score_time': array([0.17337761, 0.25275243, 0.31525551, 0.47020058, 0.35000543,
        0.24229602, 0.24193113, 0.23462313, 0.12098137, 0.86298897]),
 'param_kernel': masked_array(data=['poly', 'poly', 'poly', 'rbf', 'rbf', 'sigmoid',
                    'sigmoid', 'sigmoid', 'rbf', 'rbf'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[

In [39]:
list(a_enregistrer['params'])

[{'kernel': 'poly', 'gamma': 'scale', 'C': 10},
 {'kernel': 'poly', 'gamma': 'scale', 'C': 0.1},
 {'kernel': 'poly', 'gamma': 'scale', 'C': 50},
 {'kernel': 'rbf', 'gamma': 'scale', 'C': 1.0},
 {'kernel': 'rbf', 'gamma': 'scale', 'C': 0.1},
 {'kernel': 'sigmoid', 'gamma': 'scale', 'C': 0.1},
 {'kernel': 'sigmoid', 'gamma': 'scale', 'C': 1.0},
 {'kernel': 'sigmoid', 'gamma': 'scale', 'C': 10},
 {'kernel': 'rbf', 'gamma': 'scale', 'C': 10},
 {'kernel': 'rbf', 'gamma': 'scale', 'C': 0.01}]

In [40]:
list(a_enregistrer['mean_test_score'])

[0.5862445306043468,
 0.2652772638169182,
 0.5862885899394831,
 0.6273216739182068,
 0.45074969469754184,
 0.5179389964593328,
 0.6212415439146927,
 0.5857741933187461,
 0.6314190691235828,
 0.26267788600617786]

In [80]:
long_list = len(a_enregistrer['params'])
list =[]
for i in range(long_list):
    list.append(dict( {'model_estimator':str(random_result_SVM.estimator),'kernel':a_enregistrer['params'][i]['kernel'], 'gamma':a_enregistrer['params'][i]['gamma'], 'C' : a_enregistrer['params'][i]['C'], 'mean_test_score': a_enregistrer['mean_test_score'][i], 'std_test_score':a_enregistrer['std_test_score'][i]}))
    
list

[{'model_estimator': 'SVC()',
  'kernel': 'poly',
  'gamma': 'scale',
  'C': 10,
  'mean_test_score': 0.5862445306043468,
  'std_test_score': 0.011374414831231631},
 {'model_estimator': 'SVC()',
  'kernel': 'poly',
  'gamma': 'scale',
  'C': 0.1,
  'mean_test_score': 0.2652772638169182,
  'std_test_score': 0.0012388719650589292},
 {'model_estimator': 'SVC()',
  'kernel': 'poly',
  'gamma': 'scale',
  'C': 50,
  'mean_test_score': 0.5862885899394831,
  'std_test_score': 0.01140091322413865},
 {'model_estimator': 'SVC()',
  'kernel': 'rbf',
  'gamma': 'scale',
  'C': 1.0,
  'mean_test_score': 0.6273216739182068,
  'std_test_score': 0.009656317967873359},
 {'model_estimator': 'SVC()',
  'kernel': 'rbf',
  'gamma': 'scale',
  'C': 0.1,
  'mean_test_score': 0.45074969469754184,
  'std_test_score': 0.006032715820608887},
 {'model_estimator': 'SVC()',
  'kernel': 'sigmoid',
  'gamma': 'scale',
  'C': 0.1,
  'mean_test_score': 0.5179389964593328,
  'std_test_score': 0.006423115715018225},
 {'m

In [81]:
# formater en json
import json
json_formate = json.dumps(list)
json_formate

'[{"model_estimator": "SVC()", "kernel": "poly", "gamma": "scale", "C": 10, "mean_test_score": 0.5862445306043468, "std_test_score": 0.011374414831231631}, {"model_estimator": "SVC()", "kernel": "poly", "gamma": "scale", "C": 0.1, "mean_test_score": 0.2652772638169182, "std_test_score": 0.0012388719650589292}, {"model_estimator": "SVC()", "kernel": "poly", "gamma": "scale", "C": 50, "mean_test_score": 0.5862885899394831, "std_test_score": 0.01140091322413865}, {"model_estimator": "SVC()", "kernel": "rbf", "gamma": "scale", "C": 1.0, "mean_test_score": 0.6273216739182068, "std_test_score": 0.009656317967873359}, {"model_estimator": "SVC()", "kernel": "rbf", "gamma": "scale", "C": 0.1, "mean_test_score": 0.45074969469754184, "std_test_score": 0.006032715820608887}, {"model_estimator": "SVC()", "kernel": "sigmoid", "gamma": "scale", "C": 0.1, "mean_test_score": 0.5179389964593328, "std_test_score": 0.006423115715018225}, {"model_estimator": "SVC()", "kernel": "sigmoid", "gamma": "scale", 

In [82]:
# enregistrer dans un json
with open('monitoring_randomsearch.json', 'w') as f:
    json.dump(list,f,indent=4, ensure_ascii=False, sort_keys=False )

In [83]:
# load un json
with open('monitoring_randomsearch.json') as f:
      data = json.load(f)
for model in data:
    print(model)

{'model_estimator': 'SVC()', 'kernel': 'poly', 'gamma': 'scale', 'C': 10, 'mean_test_score': 0.5862445306043468, 'std_test_score': 0.011374414831231631}
{'model_estimator': 'SVC()', 'kernel': 'poly', 'gamma': 'scale', 'C': 0.1, 'mean_test_score': 0.2652772638169182, 'std_test_score': 0.0012388719650589292}
{'model_estimator': 'SVC()', 'kernel': 'poly', 'gamma': 'scale', 'C': 50, 'mean_test_score': 0.5862885899394831, 'std_test_score': 0.01140091322413865}
{'model_estimator': 'SVC()', 'kernel': 'rbf', 'gamma': 'scale', 'C': 1.0, 'mean_test_score': 0.6273216739182068, 'std_test_score': 0.009656317967873359}
{'model_estimator': 'SVC()', 'kernel': 'rbf', 'gamma': 'scale', 'C': 0.1, 'mean_test_score': 0.45074969469754184, 'std_test_score': 0.006032715820608887}
{'model_estimator': 'SVC()', 'kernel': 'sigmoid', 'gamma': 'scale', 'C': 0.1, 'mean_test_score': 0.5179389964593328, 'std_test_score': 0.006423115715018225}
{'model_estimator': 'SVC()', 'kernel': 'sigmoid', 'gamma': 'scale', 'C': 1.0

In [25]:
# Entrainement du Modèle optimisé 
# on doit mettre probability true sinon pas de prédict_proba possible
with mlflow.start_run():
    print("Training best model SVM...")
    svm_opti = SVC(kernel = 'rbf', C = 50, gamma = 'scale', probability=True)
    svm_opti.fit(X_train_word_features, y_train)
    pred_svm_opti = svm_opti.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_svm_opti))
    print('Precision', precision_score(y_test, pred_svm_opti ,average='weighted'))
    print('Recall', recall_score(y_test, pred_svm_opti,average='weighted'))
    print('f1-score', f1_score(y_test, pred_svm_opti,average='weighted'))
    #mlflow.sklearn.log_model(svm_opti,'SVM model')
    mlflow.log_param('algorithm','SVM')
    mlflow.log_param('best_params',svm_opti.get_params(deep=True))
    mlflow.log_metric("score",accuracy_score(y_test, pred_svm_opti))
    print("Fin de traitement")

Training best model SVM...
Accurancy 0.5902992776057792
Precision 0.5869194051648337
Recall 0.5902992776057792
f1-score 0.5785092571432933
Fin de traitement


In [26]:
file_svm = 'Flask_journaux/src/models_pickle/file_model_fitted_svm_opti.pkl' 
pickle.dump(svm_opti, open(file_svm, 'wb'))

In [53]:
svm_opti.predict_proba(test_features)

array([[2.63609685e-03, 2.71810098e-03, 4.42679652e-03, 9.90219006e-01],
       [1.69418947e-09, 1.85433148e-09, 1.29252219e-09, 9.99999995e-01],
       [6.27080273e-01, 2.18299612e-01, 1.53416654e-01, 1.20346051e-03],
       ...,
       [2.24346529e-01, 2.29862850e-01, 5.26154335e-01, 1.96362863e-02],
       [2.31603344e-01, 1.83895678e-01, 5.82938564e-01, 1.56241365e-03],
       [3.31237601e-01, 3.12897855e-01, 3.37799981e-01, 1.80645635e-02]])

In [27]:
print(classification_report(y_test, pred_svm_opti))

              precision    recall  f1-score   support

           1       0.60      0.77      0.67      6564
           2       0.52      0.40      0.45      3600
           3       0.55      0.40      0.46      3339
           4       0.89      0.73      0.80      1032

    accuracy                           0.59     14535
   macro avg       0.64      0.57      0.60     14535
weighted avg       0.59      0.59      0.58     14535



In [ ]:
# Modèle optimisé 2
with mlflow.start_run():
    print("Training best model SVM...")
    svm_opti2 = SVC(kernel = 'rbf', C = 10, gamma = 'scale',probability=True)
    svm_opti2.fit(X_train_word_features, y_train)
    pred_svm_opti2 = svm_opti2.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_svm_opti2))
    print('Precision', precision_score(y_test, pred_svm_opti2 ,average='weighted'))
    print('Recall', recall_score(y_test, pred_svm_opti2,average='weighted'))
    print('f1-score', accuracy_score(y_test, pred_svm_opti2))
    mlflow.sklearn.log_model(svm_opti2,'SVM model')
    mlflow.log_param('algorithm','SVM')
    mlflow.log_param('best_params',svm_opti2.get_params(deep=True))
    mlflow.log_metric("score",accuracy_score(y_test, pred_svm_opti2))
    print("Fin de traitement")

Training best model SVM...


In [ ]:
file_svm = 'Flask_journaux/src/models_pickle/file_model_fitted_svm_opti2.pkl' 
pickle.dump(svm_opti2, open(file_svm, 'wb'))

In [ ]:
# Modèle optimisé 3
with mlflow.start_run():
    print("Training best model SVM...")
    svm_opti3 = SVC(kernel = 'rbf', C = 1.0, gamma = 'scale',probability=True)
    svm_opti3.fit(X_train_word_features, y_train)
    pred_svm_opti3 = svm_opti3.predict(test_features)
    print('Accurancy', accuracy_score(y_test, pred_svm_opti3))
    print('Precision', precision_score(y_test, pred_svm_opti3 ,average='weighted'))
    print('Recall', recall_score(y_test, pred_svm_opti3,average='weighted'))
    print('f1-score', accuracy_score(y_test, pred_svm_opti3))
    mlflow.sklearn.log_model(svm,'SVM model')
    mlflow.log_param('algorithm','SVM')
    mlflow.log_param('best_params',svm_opti3.get_params(deep=True))
    mlflow.log_metric("score",accuracy_score(y_test, pred_svm_opti3))
    print("Fin de traitement")

In [ ]:
file_svm = 'Flask_journaux/src/models_pickle/file_model_fitted_svm_opti3.pkl' 
pickle.dump(svm_opti3, open(file_svm, 'wb'))

In [ ]:
# tuning Naive bayes seul le parametres var_smoothing peut etre tuner

## Test

In [56]:
from sklearn.model_selection import cross_val_score

cv_scoresopti = cross_val_score(svm_opti, X_train_word_features, y_train, cv=5)
cv_scoresopti2 = cross_val_score(svm_opti2, X_train_word_features, y_train, cv=5)
cv_scoresopti3 = cross_val_score(svm_opti3, X_train_word_features, y_train, cv=5)

print(cv_scoresopti)
print("cv_scoresopti mean:{}".format(np.mean(cv_scoresopti)))
print(cv_scoresopti2)
print("cv_scoresopti2 mean:{}".format(np.mean(cv_scoresopti2)))
print(cv_scoresopti3)
print("cv_scoresopti3 mean:{}".format(np.mean(cv_scoresopti3)))

[0.62312775 0.6246696  0.63053536 0.62392597 0.63207755]
cv_scoresopti mean:0.6268672484090404
[0.62334802 0.62378855 0.6294338  0.62480723 0.6311963 ]
cv_scoresopti2 mean:0.6265147769744932
[0.6154185  0.61740088 0.6311963  0.61885878 0.63141661]
cv_scoresopti3 mean:0.6228582146118854


In [73]:
results6 = knn6.fit(X_train_word_features, y_train)

file6 = 'Flask_journaux/src/models_pickle/file_model_fitted_Knn6.pkl' 
pickle.dump(results6, open(file6, 'wb'))

In [74]:
results19 = knn19.fit(X_train_word_features, y_train)

file19 = 'Flask_journaux/src/models_pickle/file_model_fitted_Knn19.pkl' 
pickle.dump(results19, open(file19, 'wb'))

In [75]:
y_hat6 = results6.predict(test_features)
y_hat_proba6 = results6.predict_proba(test_features)

In [76]:
y_hat19 = results19.predict(test_features)
y_hat_proba19 = results19.predict_proba(test_features)

In [77]:
confusion_matrix(y_test, y_hat6)

array([[1082,  714,  185,  332,   73],
       [ 624, 1194,  182,  310,  107],
       [ 420,  504, 1005,  240,   71],
       [ 599,  560,  235,  760,   64],
       [  90,  171,   69,   60, 2144]], dtype=int64)

In [78]:
confusion_matrix(y_test, y_hat19)

array([[ 962,  699,  214,  394,  117],
       [ 520, 1161,  206,  356,  174],
       [ 325,  498, 1034,  273,  110],
       [ 447,  571,  225,  878,   97],
       [  54,  134,   49,   50, 2247]], dtype=int64)

In [79]:
print('Accurancy6', accuracy_score(y_test, y_hat6))
print('Precision', precision_score(y_test, y_hat6 ,average='weighted'))
print('Recall', recall_score(y_test, y_hat6,average='weighted'))
print('f1-score', f1_score(y_test, y_hat6,average='weighted'))

Accurancy6 0.5243747350572276
Precision 0.5407635311686833
Recall 0.5243747350572276
f1-score 0.5270738032755025


In [80]:
print('Accurancy19', accuracy_score(y_test, y_hat19))
print('Precision', precision_score(y_test, y_hat19,average='weighted' ))
print('Recall', recall_score(y_test, y_hat19,average='weighted'))
print('f1-score', f1_score(y_test, y_hat19,average='weighted'))

Accurancy19 0.5325985587113183
Precision 0.5361134648890626
Recall 0.5325985587113183
f1-score 0.5308148634241694


In [34]:
print(classification_report(y_test, y_hat6))

              precision    recall  f1-score   support

           1       0.24      0.20      0.22      2406
           2       0.22      0.40      0.29      2392
           3       0.22      0.62      0.32      2302
           4       0.70      0.09      0.16      2185
           5       0.53      0.07      0.12      2192
           6       0.97      0.18      0.30      2604

    accuracy                           0.26     14081
   macro avg       0.48      0.26      0.23     14081
weighted avg       0.48      0.26      0.24     14081



In [35]:
print(classification_report(y_test, y_hat19))

              precision    recall  f1-score   support

           1       0.47      0.03      0.05      2406
           2       0.19      0.77      0.30      2392
           3       0.19      0.29      0.23      2302
           4       0.67      0.03      0.06      2185
           5       0.58      0.04      0.07      2192
           6       0.96      0.08      0.15      2604

    accuracy                           0.21     14081
   macro avg       0.51      0.21      0.14     14081
weighted avg       0.51      0.21      0.15     14081



## TEST DU MODELE

In [116]:
with open("Flask_journaux/src/models_pickle/file_model_fitted_svm_opti.pkl", 'rb') as file_model:
    lr = pickle.load(file_model)  
    print ('Model loaded')

with open("Flask_journaux/src/models_pickle/file_tfidf.pkl", 'rb') as vector:
    tfidf = pickle.load(vector)
    print ('tfidf transfo loaded')

Model loaded
tfidf transfo loaded


In [24]:
from sklearn.preprocessing import StandardScaler
titre ='Bob Mould, trente ans de distorsion'
titre_nettoyé = nettoyage(titre)
print(titre_nettoyé)
query = tfidf.transform([titre_nettoyé])
query 

 bob mould trent an distors


<1x17935 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [25]:
query.shape

(1, 17935)

In [26]:
lr.predict(query)

array([1], dtype=int64)

In [27]:
lr.predict_proba(query)

array([[0.49691023, 0.29911828, 0.11191837, 0.09205312]])

In [112]:
# dataset 2

df2= pd.read_csv('csv/data_cat2.csv')
df2

,titre,jour_publication,categorie,journal
0,"« Même si mon conjoint en a fait plus, l’essen...",2020-11-04,emploi,Le Monde
1,"Pennsylvanie, Michigan, Wisconsin... Le vote o...",2020-11-04,international,Le Monde
2,Simples spectateurs,2020-11-04,NaN,Le Monde
3,Ethiopie : le premier ministre annonce une rip...,2020-11-04,afrique,Le Monde
4,Lagardère : les clans Bolloré et Arnault négoc...,2020-11-04,economie,Le Monde
...,...,...,...,...
5496,Harry et Meghan déchus : quels membres de la f...,2021-03-03,royautes,Closer
5497,Nicolas Sarkozy : l'ancien Président explique ...,2021-03-03,politique,Closer
5498,Jahmil French : l'acteur de Degrassi : nouvell...,2021-03-03,people,Closer
5499,Meghan Markle accusée de harcèlement : les rév...,2021-03-03,royautes,Closer


In [113]:
X2 = df2.titre.map(lambda x: nettoyage(x))
y2 = df2.journal.map(lambda x: encode_journal(x))
X2

0        mêm si conjoint fait essentiel tomb dos comme...
1        pennsylvan michigan wisconsin vot ouvri blanc...
2                                        simpl spectateur
3        ethiop premi ministr annonc ripost apres atta...
4        lagarder clan bollor arnault négocient sujet ...
                              ...                        
5496     harry meghan déchus quel membr famill royal v...
5497     nicol sarkozy l'ancien président expliqu pour...
5498     jahmil french l'acteur degrass nouvel géner m...
5499     meghan markl accus harcel rével explos ancien...
5500     sylv orteg hor contrôl dan tpmp veuv ludovic ...
Name: titre, Length: 5501, dtype: object

In [117]:
X2_features = tfidf.transform(X2)
pred2 = lr.predict(X2_features)

In [118]:
print('Accurancy', accuracy_score(y2, pred2))
print('Precision', precision_score(y2, pred2 ,average='weighted'))
print('Recall', recall_score(y2, pred2,average='weighted'))
print('f1-score', f1_score(y2, pred2,average='weighted'))

Accurancy 0.4904562806762407
Precision 0.4909159620515632
Recall 0.4904562806762407
f1-score 0.48892048417856576


In [30]:
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Ludi24',
                         db='journaux')
# create cursor
cursor = connection.cursor()

query_read = "Select * from articles  where jour_publication<=date('2021-05-10') and  jour_publication>=date('2021-4-21')"
cursor.execute(query_read)
raw = cursor.fetchall()
connection.commit()

In [31]:
with open("Flask_journaux/src/models_pickle/file_model_fitted_svm_opti.pkl", 'rb') as file_model:
    lr = pickle.load(file_model)  
    print ('Model loaded')
with open("Flask_journaux/src/models_pickle/file_tfidf.pkl", 'rb') as vector:
    tfidf = pickle.load(vector)
    print ('tfidf transfo loaded')

Model loaded
tfidf transfo loaded


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [32]:
df_test = pd.DataFrame(raw, columns =["id_article","titre" ,"jour_publication" ,"categorie","journal"  ])
cursor.close()
connection.close()
df_test

,id_article,titre,jour_publication,categorie,journal
0,37371,Jérémy Frérot : l'accouchement de Laure Manaud...,2021-05-10,people,Closer
1,37372,Rachida Dati : qui a volé les clefs de son app...,2021-05-10,politique,Closer
2,37373,Maurane : le touchant hommage de Pascal Obispo...,2021-05-08,people,Closer
3,37374,Gilbert Rozon : l'ex-juré de La France a un in...,2021-05-08,people,Closer
4,37375,Renaud : cet avertissement de Lola Séchan à so...,2021-05-08,people,Closer
...,...,...,...,...,...
4171,161930,"«Time Capsule 2045», les as de la cachette",2021-05-10,arts,Libération
4172,161931,"Turner Prize, le temps des militants",2021-05-10,arts,Libération
4173,161932,Rwanda : «Nous avons changé de méthode pour tr...,2021-05-10,afrique,Libération
4174,161933,"Michel Fourniret, la fin de l’«ogre des Ardennes»",2021-05-10,police / justice,Libération


In [33]:
Xdata2 = df_test.titre.map(lambda x: nettoyage(x))
ydata2 = df_test.journal.map(lambda x: encode_journal(x))
Xdata2

0        jérémy frérot l'accouch laur manaudou deuxiem...
1                              rachid dat vol clef appart
2         mauran touch hommag pascal obispo an apres mort
3        gilbert rozon l'exjur franc incroi talent acc...
4        renaud cet avert lol séchan per refus fair va...
                              ...                        
4171                                    tim capsul cachet
4172                                 turn priz temp milit
4173                  rwand avon chang méthod traqu fugit
4174                      michel fourniret fin ogre arden
4175     grev dan centr impôt tous jour “appren contri...
Name: titre, Length: 4176, dtype: object

In [34]:
Xdata2_features = tfidf.transform(Xdata2)
preddata2 = lr.predict(Xdata2_features)

ValueError: X.shape[1] = 22618 should be equal to 17935, the number of features at training time

In [ ]:
print('Accurancy', accuracy_score(ydata2ydata2ydata2ydata2, preddata2))
print('Precision', precision_score(ydata2ydata2ydata2, preddata2 ,average='weighted'))
print('Recall', recall_score(ydata2ydata2, preddata2,average='weighted'))
print('f1-score', f1_score(ydata2, preddata2,average='weighted'))